In [20]:
from os import path
import logging

import pandas as pd

In [21]:
import sys
sys.path.append("../") # go to parent dir

In [22]:
from dependencynet.schema import SchemaBuilder
from dependencynet.model import ModelBuilder, ModelStorageService
from dependencynet.network.graphbuilder import GraphBuilder, LevelNode, ResourceNode
from dependencynet.network.stylebuilder import StyleBuilder
from dependencynet.network.graphviewer import GraphViewer
from dependencynet.network.graphml import GraphMLConverter

In [23]:
logging.basicConfig()
logger = logging.getLogger('dependencynet')
logger.setLevel(logging.INFO)

In [24]:
filename = path.join('..', 'tests', 'resources', 'data', 'compact', 'towns.csv')
data = pd.read_csv(filename, delimiter=';')
properties = ['area', 'country', 'town', 'monument']
source_df = pd.DataFrame(data, columns=properties)

In [25]:
schema_towns = SchemaBuilder().level('area', 'A').level('country', 'C').level('town', 'T') \
                              .resource('monument', 'M').render()

model = ModelBuilder().from_compact(source_df) \
                      .with_schema(schema_towns) \
                      .render()

INFO:dependencynet.schema:rendering schema
INFO:dependencynet.datasource.loaders.levelsloader:extract_items_root keys=['area'] id_pattern=A{id:02d} => shape=(2, 4)
INFO:dependencynet.datasource.loaders.levelsloader:extract_items_root keys=['area', 'country'] id_pattern={id_parent}C{id:02d} => shape=(4, 6)
INFO:dependencynet.datasource.loaders.levelsloader:extract_items_root keys=['area', 'country', 'town'] id_pattern={id_parent}T{id:02d} => shape=(5, 7)
INFO:dependencynet.datasource.loaders.resourcesloader:__extract_resource keys=['area', 'country', 'town', 'monument'] id_pattern={id_parent}M{id:02d} => shape=(4, 8)


nodes and edges

In [26]:
class AreaNode(LevelNode):
    def __init__(self, properties):
        super().__init__(properties, 'area')

In [27]:
class CountryNode(LevelNode):
    def __init__(self, properties):
        super().__init__(properties, 'country')

In [28]:
class TownNode(LevelNode):
    def __init__(self, properties):
        super().__init__(properties, 'town')

In [29]:
class MonumentNode(ResourceNode):
    def __init__(self, properties):
        super().__init__(properties, 'monument')

In [30]:
gb = GraphBuilder(model)
gb.register_node_type('area', AreaNode)
gb.register_node_type('country', CountryNode)
gb.register_node_type('town', TownNode)
gb.register_node_type('monument', MonumentNode)


gb.add_nodes_from_levels()
gb.add_nodes_from_resource('monument')
graph_model = gb.render()

INFO:dependencynet.network.graphbuilder:creating nodes for each level dataset
INFO:dependencynet.network.graphbuilder:creating edges between levels
INFO:dependencynet.network.graphbuilder:creating nodes for resource dataset monument
INFO:dependencynet.network.graphbuilder:creating edges between lower level and resource


In [31]:
#display(graph_model.G.nodes)

In [32]:
#display(graph_model.G.edges)

In [33]:
sb = StyleBuilder(schema_towns)
graph_style = sb.render()

In [34]:
import matplotlib.pyplot as plt
%matplotlib inline

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


dagre klay

In [35]:
gv = GraphViewer(graph_model)
cytoscapeobj = gv.render('dagre', graph_style, 'LR')

In [36]:
display(cytoscapeobj)

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'nodeSpacing': 10, 'edgeLenghVal': 10, 'rankDir': 'LR'}, cy…

In [37]:
filename = path.join('output', 'testyed.graphml')
GraphMLConverter(graph_model, graph_style, schema_towns).save(filename)

INFO:dependencynet.network.graphml:file saved under output/testyed.graphml


In [38]:
for node in graph_model.G.nodes(data=True):
    print(node[0])

AreaNode(classes='area level', data={'id': 'A01', 'label': 'A01 Europe'}, position={})
AreaNode(classes='area level', data={'id': 'A02', 'label': 'A02 Asia'}, position={})
CountryNode(classes='country level', data={'id': 'A01C01', 'label': 'A01C01 France'}, position={})
CountryNode(classes='country level', data={'id': 'A01C02', 'label': 'A01C02 UK'}, position={})
CountryNode(classes='country level', data={'id': 'A01C03', 'label': 'A01C03 Italia'}, position={})
CountryNode(classes='country level', data={'id': 'A02C01', 'label': 'A02C01 Japan'}, position={})
TownNode(classes='town level', data={'id': 'A01C01T01', 'label': 'A01C01T01 Paris'}, position={})
TownNode(classes='town level', data={'id': 'A01C01T02', 'label': 'A01C01T02 Lyon'}, position={})
TownNode(classes='town level', data={'id': 'A01C02T01', 'label': 'A01C02T01 London'}, position={})
TownNode(classes='town level', data={'id': 'A01C03T01', 'label': 'A01C03T01 Rome'}, position={})
TownNode(classes='town level', data={'id': 'A0